# 1.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pd.set_option('display.max_colwidth', 1000)

In [ ]:
df_train = pd.read_csv("agnews_train.csv")

In [ ]:
df_test = pd.read_csv("agnews_test.csv")

In [ ]:
# df_train = pd.read_csv("/kaggle/input/ag-news-classification-dataset/train.csv")

In [ ]:
# df_test = pd.read_csv("/kaggle/input/ag-news-classification-dataset/test.csv")

In [ ]:
df_train.head(20)

In [ ]:
df_test.head(20)

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
df_train.drop_duplicates(inplace = True)
df_test.drop_duplicates(inplace = True)

In [ ]:
df_train['text'] = df_train['Title'] + ' ' + df_train['Description']

In [ ]:
df_test['text'] = df_test['Title'] + ' ' + df_test['Description']

# 2.

In [ ]:
df_train.rename(columns = {'Class Index': 'label'}, inplace = True)
df_test.rename(columns = {'Class Index': 'label'}, inplace = True)

In [ ]:
df_train['label'] = df_train['label'] - 1 
df_test['label'] = df_test['label'] - 1
df_train['label'].value_counts()
df_train.drop(columns = ['Title', 'Description'], inplace = True)
df_test.drop(columns = ['Title', 'Description'], inplace = True)

In [ ]:
df_train.head()

In [ ]:
plt.hist(df_train['label'], edgecolor='black')
plt.xlabel('Label')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.hist(df_test['label'], edgecolor='black')
plt.xlabel('Label')
plt.ylabel('Frequency')
plt.show()

In [ ]:
from wordcloud import WordCloud

wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(df_train['text']))
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
import re

def contains_html(text):
  """Checks if a string contains HTML tags."""
  match = re.search('<.*?>', text)
  return bool(match)

html_rows = df_train[df_train['text'].apply(contains_html)]

html_rows.head()

# 3.

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from torch.optim import AdamW  


In [ ]:
#import torchtext

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# import nltk
# from nltk.tokenize import word_tokenize

In [ ]:
train_size = int(0.8 * len(df_train))

df_train_copy = df_train.copy()  
df_train = df_train_copy.iloc[:train_size]  
df_val = df_train_copy.iloc[train_size:] 

print(df_train.shape)  
print(df_val.shape)   

In [ ]:
train_tfidf_label = df_train['label'].values
train_tfidf_text = df_train['text'].values
test_tfidf_text = df_test['text'].values
test_tfidf_label = df_test['label'].values
val_tfidf_text = df_val['text'].values
val_tfidf_label = df_val['label'].values

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)

In [ ]:
X_tfidf_train = vectorizer.fit_transform(train_tfidf_text)
X_tfidf_val = vectorizer.transform(val_tfidf_text)
X_tfidf_test = vectorizer.transform(test_tfidf_text)

In [ ]:
print(X_tfidf_train.shape)
print(X_tfidf_val.shape)  
print(X_tfidf_test.shape)

In [ ]:
X_tfidf_train = X_tfidf_train.toarray()
X_tfidf_val = X_tfidf_val.toarray()
X_tfidf_test = X_tfidf_test.toarray()

In [ ]:
print(vectorizer.get_feature_names_out()) 

In [ ]:
X_tfidf_train, X_tfidf_train[0:1].shape

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
train_texts = df_train['text'].tolist() 
train_tokens = [tokenizer.tokenize(text) for text in train_texts]

In [ ]:
val_texts = df_val['text'].tolist()
val_tokens = [tokenizer.tokenize(text) for text in val_texts]

In [ ]:
test_texts = df_test['text'].tolist()
test_tokens = [tokenizer.tokenize(text) for text in test_texts]

In [ ]:
train_encode = [torch.tensor(tokenizer.encode(text, truncation=True, max_length=256)) for text in df_train['text']]
val_encode = [torch.tensor(tokenizer.encode(text, truncation=True, max_length=256)) for text in df_val['text']]
test_encode = [torch.tensor(tokenizer.encode(text, truncation=True, max_length=256)) for text in df_test['text']]

In [ ]:
x_train = pad_sequence(train_encode, batch_first=True, padding_value=tokenizer.pad_token_id)
x_val = pad_sequence(val_encode, batch_first=True, padding_value=tokenizer.pad_token_id)
x_test = pad_sequence(test_encode, batch_first=True, padding_value=tokenizer.pad_token_id)

In [ ]:
print(x_train.shape) 

In [ ]:
train_label = df_train['label'].tolist()
val_label = df_val['label'].tolist()
test_label = df_test['label'].tolist()

In [ ]:
train_label[:10]

In [ ]:
y_train = torch.tensor(train_label, dtype=torch.long)
y_val = torch.tensor(val_label, dtype=torch.long)
y_test = torch.tensor(test_label, dtype=torch.long)

In [ ]:
x_train, y_train, x_test, y_test, x_val, y_val

# 4.

In [ ]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.metrics import log_loss

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
lr = SGDClassifier(loss = "log_loss", max_iter=1, warm_start = True, learning_rate="constant", eta0=0.01, random_state=42)

best_accuracy = 0
no_improvement_count = 0
early_stopping_threshold = 5
epochs = 50

In [ ]:
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(epochs):
    lr.fit(X_tfidf_train, train_tfidf_label)

    train_preds = lr.predict(X_tfidf_train)
    val_preds = lr.predict(X_tfidf_val)

    train_probs = lr.predict_proba(X_tfidf_train)
    val_probs = lr.predict_proba(X_tfidf_val)

    train_loss = log_loss(train_tfidf_label, train_probs)
    val_loss = log_loss(val_tfidf_label, val_probs)

    train_accuracy = np.mean(train_preds == train_tfidf_label)
    val_accuracy = np.mean(val_preds == val_tfidf_label)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} | Train Acc: {train_accuracy:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_accuracy:.4f}")    
    
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        no_improvement_count = 0
    else:
        no_improvement_count += 1
    
    if no_improvement_count >= early_stopping_threshold:
        print("Early stopping triggered.")
        break

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(range(1, len(train_losses) + 1), train_losses, label="Train Loss")
plt.plot(range(1, len(val_losses) + 1), val_losses, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Train vs Validation Loss")
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(range(1, len(train_accuracies) + 1), train_accuracies, label="Train Accuracy")
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Train vs Validation Accuracy")
plt.legend()
plt.grid()
plt.show()

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_tfidf_train, train_tfidf_label)

In [ ]:
rfc_preds_test = rfc.predict(X_tfidf_test)
rfc_preds_val = rfc.predict(X_tfidf_val)

test_acc = np.mean(rfc_preds_test == test_tfidf_label)
val_acc = np.mean(rfc_preds_val == val_tfidf_label)

print(f"RF - Acc: {test_acc:.4f} | Val Acc: {val_acc:.4f}")

In [ ]:
xgb = XGBClassifier(n_estimator = 1000, max_depth = 6, learning_rate = 0.1, eval_metric='mlogloss', random_state=42)

In [ ]:
eval_set = [(X_tfidf_train, train_tfidf_label), (X_tfidf_val, val_tfidf_label)]
xgb.fit(X_tfidf_train, train_tfidf_label, eval_set=eval_set, verbose=False)

In [ ]:
results = xgb.evals_result()
train_loss = results['validation_0']['mlogloss']
val_loss = results['validation_1']['mlogloss']

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(train_loss, label="Train Loss")
plt.plot(val_loss, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("XGBoost - Train vs Validation Loss")
plt.legend()
plt.grid()
plt.show()

In [ ]:
class TextDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.labels[idx]

In [ ]:
batch_size = 32

In [ ]:
train_dataset = TextDataset(x_train, y_train)
val_dataset = TextDataset(x_val, y_val)
test_dataset = TextDataset(x_test, y_test)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embd_dim, hidden_dim, output_dim, num_layers = 1):
        super(LSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embd_dim)
        self.lstm = nn.LSTM(embd_dim, hidden_dim, num_layers, batch_first=True)
        self.dropout = nn.Dropout(0.25)
        self.dense = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        last_hidden = self.dropout(lstm_out[:, -1,  :])
        output = self.dense(last_hidden)
        return output

In [ ]:
vocab_size = tokenizer.vocab_size
embd_dim = 256
hidden_dim = 512
output_dim = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
lstm = LSTM(vocab_size, embd_dim, hidden_dim, output_dim).to(device)

optimizer = torch.optim.Adam(lstm.parameters(), lr = 1e-5, weight_decay=1e-4)

criterion = torch.nn.CrossEntropyLoss()

In [ ]:
def evaluate(model, data_loader, criterion, device):
    model.eval()
    predictions = []
    targets = []
    total_loss = 0
    with torch.no_grad():
        for batch in data_loader:
            texts, labels = batch  
            texts, labels = texts.to(device), labels.to(device)  

            outputs = model(texts)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            predictions.append(outputs.argmax(dim=1))
            targets.append(labels)
    
    predictions = torch.cat(predictions)
    targets = torch.cat(targets)
    accuracy = (predictions == targets).float().mean().item()
    avg_loss = total_loss / len(data_loader)

    return avg_loss, accuracy

In [ ]:
num_epochs = 50
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

patience = 3
best_val_accuracy = 0
no_improvement_count = 0

for epoch in range(num_epochs):
    epoch_loss = []
    lstm.train()
    total_loss = 0
    for texts, labels in train_loader:
        texts, labels = texts.to(device), labels.to(device)  
        optimizer.zero_grad()
        outputs = lstm(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    train_loss, train_accuracy = evaluate(lstm, train_loader, criterion, device)
    val_loss, val_accuracy = evaluate(lstm, val_loader, criterion, device)

    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

    if val_accuracy < best_val_accuracy:
        best_val_accuracy = val_accuracy
        no_improvement_count = 0
        best_model = lstm.state_dict()
    else:
        no_improvement_count += 1
        if no_improvement_count >= patience:
            print(f"Early stopping triggered at epoch {epoch+1}.")
            break

lstm.load_state_dict(best_model)

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(range(1, num_epochs+1), train_losses, 'g-', label='Training Loss')
ax1.plot(range(1, num_epochs+1), val_losses, 'r-', label='Validation Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color='g')
plt.title('Training Loss Over Epochs')
ax1.legend()
plt.grid()
plt.show()

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(range(1, num_epochs+1), train_accuracies, 'b-', label='Training Accuracy')
ax1.plot(range(1, num_epochs+1), val_accuracies, 'y-', label='Validation Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy', color='b')
plt.title('Test Accuracy Over Epochs')
ax1.legend()
ax1.grid()
plt.show()

In [ ]:
from transformers import RobertaForSequenceClassification

In [ ]:
model_name = "roberta-base"
num_labels = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=1e-4)

criterion = torch.nn.CrossEntropyLoss()

In [ ]:
def evaluate(model, data_loader, criterion, device):
    model.eval()
    predictions = []
    targets = []
    total_loss = 0

    with torch.no_grad():
        for batch in data_loader:
            inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
            labels = batch["labels"].to(device)

            outputs = model(**inputs)
            loss = criterion(outputs.logits, labels)
            total_loss += loss.item()

            predictions.append(outputs.logits.argmax(dim=1))
            targets.append(labels)

    predictions = torch.cat(predictions)
    targets = torch.cat(targets)
    accuracy = (predictions == targets).float().mean().item()
    avg_loss = total_loss / len(data_loader)

    return avg_loss, accuracy


In [ ]:
num_epochs = 50
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

patience = 3
best_val_accuracy = 0
no_improvement_count = 0

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()

        inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
        labels = batch["labels"].to(device)

        outputs = model(**inputs)
        loss = criterion(outputs.logits, labels)

        loss.backward()
        optimizer.step()

    train_loss, train_accuracy = evaluate(model, train_loader, criterion, device)
    val_loss, val_accuracy = evaluate(model, val_loader, criterion, device)

    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        no_improvement_count = 0
        best_model_state = model.state_dict()  
    else:
        no_improvement_count += 1
        if no_improvement_count >= patience:
            print(f"Early stopping triggered at epoch {epoch+1}.")
            break

model.load_state_dict(best_model_state)
